In [1]:
%%capture
import os
import site
site.addsitedir('/home/schirrmr/.local/lib/python2.7/site-packages/')
site.addsitedir('/usr/lib/pymodules/python2.7/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
%cd /home/schirrmr/braindecode/code/braindecode/
assert 'THEANO_FLAGS' in os.environ
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'
%load_ext autoreload
%autoreload 2
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

import matplotlib.lines as mlines
import seaborn
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7
seaborn.set_style('darkgrid')

from braindecode.scripts.print_results import ResultPrinter
from braindecode.csp.print_results import CSPResultPrinter
import logging
log = logging.getLogger()
log.setLevel("DEBUG")

In [2]:
%%javascript
$('body').append('<script src="https://dl.dropboxusercontent.com/u/34637013/work/ipython-notebooks/js-for-table-sorting/jquery.tablesorter.min.js"></script>')
// Sort tables every 5 seconds...
var makeTablesSortable;
makeTablesSortable = function() {
  $('.text_cell_render table').not('.sorted_table').tablesorter().addClass('sorted_table');
}

setInterval(makeTablesSortable, 5000);

<IPython.core.display.Javascript object>

## Basic Model

This is the network structure. In my multiple prediction implementation, when stride is used to half the dimension, the data is moved to the trial axis, not removed as normally would happen. So structure here does not show the whole network, but gives the basic idea.

I don't show batch normalization layers here.

|Layer/Block|#Layers|Units|Filter Size|Nonlinearity/PoolingMode|Output Size|
|-|
|Input|-|22|-|-|1000x1|
|Conv|1|24|(22)x3x1|Relu/Elu|1000x1|
|ResBlocks|2/3|24|3x1|Relu/Elu|1000x1|
|ResBlock|1|48|3x1|Relu/Elu|500x1|
|ResBlocks|2/3|48|3x1|Relu/Elu|500x1|
|ResBlock|1|72|3x1|Relu/Elu|250x1|
|ResBlock|2/3|72|3x1|Relu/Elu|250x1|
|ResBlock|1|72|3x1|Relu/Elu|125x1|
|ResBlock|2/3|72|3x1|Relu/Elu|125x1|
|ResBlock|1|72|3x1|Relu/Elu|63x1|
|ResBlock|2/3|72|3x1|Relu/Elu|63x1|
|ResBlock|1|72|3x1|Relu/Elu|32x1|
|ResBlock|2/3|72|3x1|Relu/Elu|32x1|
|ResBlock|1|72|3x1|Relu/Elu|16x1|
|Pool|1|-|(8/10)x1|Mean|(~7)x1|
|Conv|1|-|1x1|Softmax|7x1|

Then afterwards reshape to get all predictions.

## Full Results all datasets

Note, Deep5 reaches around 71% on BCI Competition.



|id|n_layer_per_block|final_pool_len|time|std|test_sample|std|test|std|train_sample|std|train|std|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|0|2|10|0:55:53|0:21:36|60.75%|11.55%|63.19%|12.31%|99.97%|0.08%|100.00%|0.00%|
|3|3|10|1:07:01|0:23:15|58.82%|12.66%|61.54%|13.07%|99.90%|0.16%|100.00%|0.00%|
|6|2|8|0:49:34|0:12:13|55.83%|13.31%|59.07%|14.82%|99.61%|0.58%|99.92%|0.22%|
|7|2|9|0:45:48|0:09:53|58.68%|13.58%|61.57%|14.62%|99.55%|1.28%|99.85%|0.44%|

## Exploration only on Dataset 3

In [13]:
from braindecode.results.results import ResultPool, get_final_misclasses
import pandas as pd
from copy import deepcopy
from braindecode.analysis.pandas_util import (load_data_frame, remove_dollar, pairwise_compare_frame,
    to_numeric_where_possible, round_numeric_columns)
import pandas as pd

In [22]:
data_frame = load_data_frame('data/models/paper/bci-competition/cnt/resnet/', params=dict(whisker_percent=None,
    dataset_filename='data/bci-competition-iv/2a/A03T.mat'))


# Remove old buggy experiments
data_frame = data_frame[data_frame.nonlinearity != '-']

Note, deep5 reaches around 90% on this dataset.

Comparing different values for one parameter at a time when the remaining parameters are fixed.

* n_exp is number of experiments

In [16]:
pairwise_compare_frame(data_frame)

,n_exp,val_1,val_2,acc_1,acc_2,diff,std,perm
first_filter_length,4,3,11,66.3,59.7,-6.6,6.1,12.5
nonlinearity,4,relu,elu,67.0,79.2,12.2,6.3,6.2
n_layers_per_block,10,2,3,74.3,69.6,-4.7,6.0,1.4
final_pool_length,5,8,10,67.8,65.6,-2.2,2.3,9.4
final_pool_length,1,8,9,82.3,82.3,0.0,0.0,50.0
final_pool_length,1,10,9,80.6,82.3,1.7,0.0,50.0
l2_factor,2,0.0001,0.001,80.7,78.5,-2.3,2.6,50.0
l2_factor,2,0.0001,1e-05,80.7,80.9,0.2,0.5,50.0
l2_factor,2,0.001,1e-05,78.5,80.9,2.4,2.1,25.0
max_epochs,2,150,500,80.0,80.7,0.7,0.7,50.0


## Best 10 Experiments

In [12]:
rounded_frame = round_numeric_columns(data_frame,1)
rounded_frame.sort_values(by='accuracy',ascending=False)[:10]

,projection,l2_factor,max_epochs,final_pool_length,first_filter_length,nonlinearity,n_layers_per_block,n_first_filters,accuracy
37,False,0,500,9,3,elu,2,24,82.3
36,False,0,500,8,3,elu,2,24,82.3
32,False,0,500,10,3,elu,2,24,81.2
33,False,0,500,10,3,elu,3,24,80.9
31,False,0,500,10,3,elu,2,24,80.9
35,False,0,500,10,3,elu,3,24,80.6
25,False,0,150,10,3,elu,2,24,80.6
30,False,0,500,10,3,elu,2,24,80.6
24,False,0,150,10,3,elu,3,24,79.5
29,True,0,150,10,3,elu,3,24,79.2
